In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/bitext-retail-llm-chatbot-traning-dataset/bitext-retail-banking-llm-chatbot-training-dataset.csv


In [2]:
!pip install transformers datasets torch

In [3]:
pip install --upgrade pip

Note: you may need to restart the kernel to use updated packages.


In [4]:
pip uninstall -y tensorflow

Note: you may need to restart the kernel to use updated packages.


In [5]:
pip install tensorflow-cpu

Note: you may need to restart the kernel to use updated packages.


In [6]:
!pip install ipywidgets


# All Imports

In [7]:
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import T5Tokenizer, T5ForConditionalGeneration, get_scheduler
from sklearn.model_selection import train_test_split
from torch.cuda.amp import GradScaler, autocast
from tqdm import tqdm
import os

W1207 17:03:49.628000 137842462037696 torch/_inductor/compile_worker/subproc_pool.py:126] SubprocPool unclean exit


In [8]:
df = pd.read_csv(r'/kaggle/input/bitext-retail-llm-chatbot-traning-dataset/bitext-retail-banking-llm-chatbot-training-dataset.csv')
df

,tags,instruction,category,intent,response
0,BCIPZ,"I would like to acivate a card, can you help me?",CARD,activate_card,I'm here to assist you with that! Activating y...
1,BCILZ,"I have to activate an Visa online, how can I d...",CARD,activate_card,I'm here to assist you with activating your {{...
2,BCIPQZ,I'd like to actiate a card where do i do it,CARD,activate_card,I can help you with that! Activating your card...
3,BCLPQZ,I'd likke to activate a visa on mobile i need ...,CARD,activate_card,I'm here to assist you with activating your {{...
4,BCILPZ,"I would ilke to activate a credit card online,...",CARD,activate_card,I'm here to assist you with activating your cr...
...,...,...,...,...,...
25540,BCEQZ,I'm trying to find apassword i need help,PASSWORD,set_up_password,I'm here to assist you in finding and setting ...
25541,BCIPQ,im looking for a password will uhelp me,PASSWORD,set_up_password,I'm here to assist you in setting up a passwor...
25542,BCIPZ,"I'd like to set up a password, will you help me?",PASSWORD,set_up_password,I'm here to assist you in setting up your pass...
25543,BCQ,wanna set up a password i need assistance,PASSWORD,set_up_password,I'm here to assist you in setting up your pass...


In [9]:
df['input_text'] = "<banking_query> Query: " + df['instruction'] + ' | Category: ' + df['category']
df['target_text'] = df['response']

In [10]:
df.head()

,tags,instruction,category,intent,response,input_text,target_text
0,BCIPZ,"I would like to acivate a card, can you help me?",CARD,activate_card,I'm here to assist you with that! Activating y...,<banking_query> Query: I would like to acivate...,I'm here to assist you with that! Activating y...
1,BCILZ,"I have to activate an Visa online, how can I d...",CARD,activate_card,I'm here to assist you with activating your {{...,<banking_query> Query: I have to activate an V...,I'm here to assist you with activating your {{...
2,BCIPQZ,I'd like to actiate a card where do i do it,CARD,activate_card,I can help you with that! Activating your card...,<banking_query> Query: I'd like to actiate a c...,I can help you with that! Activating your card...
3,BCLPQZ,I'd likke to activate a visa on mobile i need ...,CARD,activate_card,I'm here to assist you with activating your {{...,<banking_query> Query: I'd likke to activate a...,I'm here to assist you with activating your {{...
4,BCILPZ,"I would ilke to activate a credit card online,...",CARD,activate_card,I'm here to assist you with activating your cr...,<banking_query> Query: I would ilke to activat...,I'm here to assist you with activating your cr...


In [11]:
train_data, test_data = train_test_split(df, test_size=0.1, random_state=42)
train_data, val_data = train_test_split(train_data, test_size=0.1, random_state=42)

In [12]:
train_data = train_data.sample(frac=0.5, random_state=42)
val_data = val_data.sample(frac=0.5, random_state=42)

In [13]:
class chatbotDataset(Dataset):
    def __init__(self, dataframe, tokenizer, max_len=256):
        self.data = dataframe
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        row = self.data.iloc[index]
        input_text = row['input_text']
        target_text = row['target_text']

        input_encoding = self.tokenizer(
            input_text,
            max_length=self.max_len,
            padding='max_length',
            truncation=True,
            return_tensors='pt'
        )

        target_encoding = self.tokenizer(
            target_text,
            max_length=self.max_len,
            padding='max_length',
            truncation=True,
            return_tensors='pt'
        )

        return {
            "input_ids": input_encoding["input_ids"].squeeze(0),
            "attention_mask": input_encoding["attention_mask"].squeeze(0),
            "labels": target_encoding["input_ids"].squeeze(0)
        }


In [14]:
!pip install sentencepiece

In [15]:
model_name = 'google/flan-t5-small'
tokenizer = T5Tokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name)

tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


config.json:   0%|          | 0.00/1.40k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/308M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [16]:
train_dataset = chatbotDataset(train_data, tokenizer, max_len=256)
val_dataset = chatbotDataset(val_data, tokenizer, max_len=256)
train_loader = DataLoader(train_dataset, batch_size=4, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=4)

In [17]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

T5ForConditionalGeneration(
  (shared): Embedding(32128, 512)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 512)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=512, out_features=384, bias=False)
              (k): Linear(in_features=512, out_features=384, bias=False)
              (v): Linear(in_features=512, out_features=384, bias=False)
              (o): Linear(in_features=384, out_features=512, bias=False)
              (relative_attention_bias): Embedding(32, 6)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseGatedActDense(
              (wi_0): Linear(in_features=512, out_features=1024, bias=False)
              (wi_1): Linear(in_features=512, out_features=1024, bias=False)
              (wo): 

In [18]:
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-4)
num_training_steps = len(train_loader) * 2  
lr_scheduler = get_scheduler("linear", optimizer=optimizer, num_warmup_steps=0, num_training_steps=num_training_steps)

In [20]:
scaler = torch.amp.GradScaler('cuda')

In [21]:
save_directory = "./fine_tuned_flan_t5_small"
os.makedirs(save_directory, exist_ok=True)

In [22]:
def validate_model(model, val_loader, device):
    model.eval()
    total_loss = 0
    with torch.no_grad():
        for batch in val_loader:
            input_ids = batch["input_ids"].to(device)
            attention_mask = batch["attention_mask"].to(device)
            labels = batch["labels"].to(device)

            with autocast():
                outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
                loss = outputs.loss

            total_loss += loss.item()

    avg_loss = total_loss / len(val_loader)
    print(f"Validation Loss: {avg_loss}")

In [23]:
def train_model(model, train_loader, val_loader, optimizer, lr_scheduler, device, epochs=2):
    model.train()
    for epoch in range(epochs):
        print(f"Epoch {epoch + 1}/{epochs}")
        total_loss = 0
        loop = tqdm(train_loader, leave=True)
        for batch in loop:
            optimizer.zero_grad()
            input_ids = batch["input_ids"].to(device)
            attention_mask = batch["attention_mask"].to(device)
            labels = batch["labels"].to(device)

            with autocast():  
                outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
                loss = outputs.loss

            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()
            lr_scheduler.step()

            total_loss += loss.item()
            loop.set_description(f"Epoch {epoch + 1}")
            loop.set_postfix(loss=loss.item())

        avg_loss = total_loss / len(train_loader)
        print(f"Train Loss: {avg_loss}")
        validate_model(model, val_loader, device)

In [24]:
train_model(model, train_loader, val_loader, optimizer, lr_scheduler, device, epochs=2)

model.save_pretrained(save_directory)
tokenizer.save_pretrained(save_directory)

Epoch 1/2


  0%|          | 0/2587 [00:00<?, ?it/s]/tmp/ipykernel_328/36946622.py:13: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():
Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.
Epoch 1: 100%|██████████| 2587/2587 [42:20<00:00,  1.02it/s, loss=0.739]
/tmp/ipykernel_328/1591848092.py:10: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():


Train Loss: 1.2647752278921085
Validation Loss: 0.806526797513167
Epoch 2/2


Epoch 2: 100%|██████████| 2587/2587 [56:51<00:00,  1.32s/it, loss=0.441] 


Train Loss: 0.7051361844686638
Validation Loss: 0.6923341222314371


('./fine_tuned_flan_t5_small/tokenizer_config.json',
 './fine_tuned_flan_t5_small/special_tokens_map.json',
 './fine_tuned_flan_t5_small/spiece.model',
 './fine_tuned_flan_t5_small/added_tokens.json')

In [3]:
def generate_response(model, tokenizer, input_text, device):
    model.eval()
    input_ids = tokenizer(
        input_text, return_tensors="pt", truncation=True, max_length=256
    ).input_ids.to(device)

    outputs = model.generate(
        input_ids,
        max_length=150,
        num_beams=5,
        no_repeat_ngram_size=2,
        temperature=0.7,
        top_k=50,
        top_p=0.95
    )

    return tokenizer.decode(outputs[0], skip_special_tokens=True)